In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Installing the NetworkX Library to work with Network Graphs & their Visualizations
# !pip install networkx

### Importing & Shaping Libraries, Modules & Datasets

In [ ]:
# The NetworkX Library is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks
import networkx as nx
# The Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language
import pandas as pd
# The Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python
import matplotlib.pyplot as plt
# The Random Module is a built-in Python Module to work with Random Numbers
import random
import copy
from collections import defaultdict
import time
import math

In [ ]:
# Converting the Dataframe structure into an Edgelist to feed into the NetworkX Library function
edges = pd.read_csv("com-amazon.ungraph.txt", sep="\t")
# edges = pd.read_csv("LFR_500_0.8_edge_list.txt", sep=" ")

# edges = pd.read_csv("CA-GrQc.txt", sep="\t")

edges

In [ ]:
# Creating the NetworkX Graph using Pandas Edgelist
GOT = nx.from_pandas_edgelist(edges, source = 'From', target = 'To')
# Dictionary storing the Degree of each node in the Graph
weighted_degrees = dict(nx.degree(GOT))

In [ ]:
# Sort node names based on your desired range
current_names = list(sorted(GOT.nodes()))

# Sort node names based on your desired range
num_nodes = len(current_names)
sorted_names = list(range(0, num_nodes))

# Create a mapping from current names to sorted names
mapping = {current_names[i]: sorted_names[i] for i in range(num_nodes)}

# Rename nodes
nx.relabel_nodes(GOT, mapping, copy=False)

# Now your graph nodes are renamed from 1 to the number of nodes
# You can print the nodes to verify
print(sorted(GOT.nodes()))
weighted_degrees = dict(nx.degree(GOT))

#### Objective Function

In [ ]:
def sort_nodes(nodes, centrality):
    """
    Sorts the Nodes in the given list of nodes based on the corresponding Centrality Measure of each node
    Parameters:
        nodes = List of nodes which has to be sorted
        centrality = List of Centrality Measures corresponding to each node in the list
    Returns:
        List of nodes sorted on the basis of their Centrality Measure in descending order
    """
    result = sorted(list(zip(centrality, nodes)), reverse = True)
    result = zip(*result)
    result = [list(tuple) for tuple in result]
    return result[1]

def sort_degree_centrality(nodes):
    """
    Calculates the Degree Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return cen_measure

def sort_eigenvector_centrality(nodes):
    """
    Calculates the Eigenvector Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_betweenness_centrality(nodes):
    """
    Calculates the Betweenness Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.betweenness_centrality(GOT)
    return centrality

def sort_closeness_centrality(nodes):
    """
    Calculates the Closeness Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.closeness_centrality(GOT)
    return centrality

def sort_katz_centrality(nodes):
    """
    Calculates the Katz Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_percolation_centrality(nodes):
    """
    Calculates the Percolation Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)





In [ ]:
def one_hop_area(meme):

    temp = [] # Temporary list to store the One-Hop Area nodes of the meme
    for i in meme:
        temp.extend(list(GOT.neighbors(i)))
    return list(set(temp) - set(meme)) # Returning only unique nodes, to avoid Node Repetition

def two_hop_area(meme):

    one_hop = one_hop_area(meme)
    temp = [] # Temporary list to store the Two-Hop Area nodes of the meme
    for i in one_hop:
        temp.extend(list(GOT.neighbors(i)))
    return list(set(temp) - set(meme) - set(one_hop)) # Returning only unique nodes, to avoid Node Repetition

def calc_pcm_prob(nodes):
    prob = [] # Temporary List to store the corresponding Cascade Probability of the Nodes
    for i in nodes:
        prob.append(weighted_degrees[i] / GOT.number_of_nodes())
    return prob

def calc_edges(group1, group2):
    count = [] # Temporary storage to store the Number of edges corresponding each node
    for i in group2:
        edges = list(nx.edges(GOT, nbunch = [i]))
        temp = 0
        for i in edges:
            if (i[1] in group1) or (i[1] in group2):
                temp += 1
        count.append(temp)
    return count

def sum_pd(list1, list2):
    p = 0 # Temporary variable to store the Sum-Product of the two lists
    for i in range(len(list1)):
        p += (list1[i] * list2[i])
    return p

def calc_edge_prob(meme, one_hop):
    N = GOT.number_of_nodes() # Total Nodes in the Graph
    prob_sum_bet = 0 # Temporary variable to store the sum of the Edge Probabilities
    prob_sum_cls = 0
    for i in one_hop:
        prob_prod = 1
        for j in meme:
            pij = 0.01
            pij += ((GOT.degree(i) + GOT.degree(j)) / N)
            pij += (len(list(nx.common_neighbors(GOT, i, j))) / N)
            prob_prod *= (1 - pij)
        prob_sum_bet += (1 - prob_prod)*bet_cen[i]
        prob_sum_cls += (1 - prob_prod)*cls_cen[i]
    return max(prob_sum_bet,prob_sum_cls)

def LIE(meme):
    k=len(meme)
    Ns1_S = one_hop_area(meme) # One-Hop area of the Meme
    Ns2_S = two_hop_area(meme) # Two-Hop Area of the Meme
    pu = calc_pcm_prob(Ns2_S)
    du = calc_edges(Ns1_S, Ns2_S)
    return  k + ((1 + ((1 / len(Ns1_S)) * sum_pd(pu, du))) * calc_edge_prob(meme, Ns1_S)) if len(Ns1_S) !=0 else 0




In [ ]:

def pareto_dominance(seed_set_x, seed_set_y):
    """Check if seed set x dominates seed set y."""
    all_less_equal = seed_set_x['F'] >=seed_set_y['F'] and seed_set_x['sigma'] >= seed_set_y['sigma']
    any_strictly_greater = seed_set_x['F'] > seed_set_y['F'] or seed_set_x['sigma'] > seed_set_y['sigma']
    return all_less_equal and any_strictly_greater

def pareto_optimal_seed_sets(seed_sets):
    """Identify Pareto-optimal seed sets from a list of seed sets."""
    pareto_optimal_sets = []
    for seed_set_x in seed_sets:
        if not any(pareto_dominance(seed_set_y, seed_set_x) for seed_set_y in seed_sets if seed_set_x != seed_set_y):
            pareto_optimal_sets.append(seed_set_x)
    return pareto_optimal_sets

def pareto_rank(seed_sets):
    """Assign Pareto rank to seed sets."""
    ranks = [0] * len(seed_sets)
    current_rank = 0
    while True:
        pareto_sets = []
        for i, seed_set in enumerate(seed_sets):
            if ranks[i] != 0:
                continue  # Already assigned a rank
            if all(not pareto_dominance(seed_set, other_set) for j, other_set in enumerate(seed_sets) if i != j and ranks[j] == 0):
                pareto_sets.append(seed_set)
                ranks[i] = current_rank + 1
        if not pareto_sets:
            break
        current_rank += 1
    for i in range(len(seed_sets)):
      seed_sets[i]['rank'] = ranks[i]


def crowding_distance(seed_sets):
    """Calculate crowding distance for seed sets based on their objectives."""
    for seed_set in seed_sets:
        seed_set['distance'] = 0
    seed_sets.sort(key=lambda x: x['F'])
    seed_sets[0]['distance'] = float('inf')
    seed_sets[len(seed_set)-1]['distance'] = float('inf')
    rnge = seed_sets[len(seed_sets)-1]['F'] - seed_sets[0]['F']
    for i in range(1,len(seed_sets)-1):
      if rnge != 0:
        seed_sets[i]['distance'] += (seed_sets[i+1]['F'] - seed_sets[i-1]['F'])/ rnge
      else:
        seed_sets[i]['distance'] = float('inf')

    seed_sets.sort(key = lambda x: x['sigma'])
    seed_sets[0]['distance'] = float('inf')
    seed_sets[len(seed_sets)-1]['distance'] = float('inf')
    rnge = seed_sets[len(seed_sets)-1]['sigma'] - seed_sets[0]['sigma']
    for i in range(1,len(seed_sets)-1):
      if rnge != 0:
        seed_sets[i]['distance'] += (seed_sets[i+1]['sigma'] - seed_sets[i-1]['sigma'])/ rnge
      else:
        seed_sets[i]['distance'] = float('inf')


def select_seed_sets(candidate_sets, Q):
    """Select Q seed sets from candidate sets."""
    selected = []
    print(candidate_sets)
    candidate_sets.sort(key=lambda x: (-x['rank'], x['distance']))
    selected = candidate_sets[:Q]
    return selected



In [ ]:
import collections


def calculate_gini_coefficient(network, seed_set, m):
    """Calculate Gini coefficient for a given seed set."""
    # Perform diffusion process to get influenced nodes
    influenced_nodes = simulate_diffusion(network, seed_set)

    # Calculate fractions of influenced nodes in each group
    fractions = [len(set(group) & set(influenced_nodes)) / len(group) for group in groups]

    # Sort fractions
    sorted_fractions = sorted(fractions)

    # Calculate Gini coefficient
    n = len(sorted_fractions)
    gini = (np.sum([(2 * i - n + 1) * sorted_fractions[i] for i in range(n)]) / (n * np.sum(sorted_fractions)))

    return gini

# def calculate_influence_spread(network, seed_set):
#     influence = []
#     for i in range(network.number_of_nodes()):
#       if i in seed_set:
#         influence.append(1)
#       else:
#         influence.append(0)
#     influence = simulate_diffusion(network, influence)
#     count = 0
#     for i in range(len(influence)):
#       count += influence[i]
#     return count

def calculate_influence_spread(network, seed_set):
#     influence = []
#     for i in range(network.number_of_nodes()):
#       if i in seed_set:
#         influence.append(1)
#       else:
#         influence.append(0)
    influence = simulate_diffusion(network, seed_set)
#     count = 0
#     for i in range(len(influence)):
#       count += influence[i]
    return len(influence)

def calculate_F(network, seed_set):
    """Calculate fairness metric F(S) for a given seed set."""
    gini_coefficient = calculate_gini_coefficient(network, seed_set,0.5)
    return 1 - gini_coefficient

def calculate_sigma(network, seed_set):
    """Calculate influence spread sigma(S) for a given seed set."""
    return LIE(seed_set)

def seed_set_evaluation(network, seed_sets):
    """Evaluate F(S) and sigma(S) for a list of seed sets."""
    info = []
    for seed_set in seed_sets:
        F = calculate_F(network, seed_set)
        sigma = calculate_sigma(network, seed_set)
        inf_spr = calculate_influence_spread(network,seed_set)
        info.append({'seed':seed_set,'F':F,'sigma':sigma, 'spread':inf_spr})
    return info

In [ ]:
bet_cen = nx.betweenness_centrality(GOT)
cls_cen = nx.closeness_centrality(GOT)
# bet_cen=cls_cen.copy()

In [ ]:
with open("SOC_Amazon_Closeness_centrality.txt", "w") as f:
    for node, centrality in cls_cen.items():
        f.write(f"Node {node}: {centrality}\n")

In [ ]:
import numpy as np
import copy
def levy_walk(position, scale=0.01, beta=1.5):
    u = np.random.normal()
    v = np.random.normal()
    s = u / (np.linalg.norm(v) ** (1/beta))
    stepsize = scale * s

    random_numbers = np.random.standard_cauchy()

    step = stepsize * position
    levy_walk_step = position + step * random_numbers

    return levy_walk_step

def initialize_particle(npop,k):
    position=[]
    for i in range(0,npop):
      position.append(random.sample(range(GOT.number_of_nodes()),k))
    return position

def probabilityupdate(action_probabilities, chosen_action, signal, alpha, beta):
    if signal == 1:  # If action was successful
        action_probabilities[chosen_action] += alpha * (1 - action_probabilities[chosen_action])
        action_probabilities[1 - chosen_action] = (1 - alpha) * action_probabilities[1 - chosen_action]
    else:  # If action failed
        action_probabilities[chosen_action] = (1 - beta) * action_probabilities[chosen_action]
        action_probabilities[1 - chosen_action] = beta + (1 - beta) * action_probabilities[1 - chosen_action]

    return action_probabilities

def update_position(particle, destination, num_nodes, iteration, max_iter, a, action_probabilities, signal):
    position = particle
    r1 = a * (1 - iteration / max_iter)
    r2 = 2 * np.pi * random.random()
    r3 = 2 * random.random()
    present_nodes = set()
    not_present_nodes = set(GOT.nodes() - present_nodes)

    chosen_action =  np.random.choice([0, 1], p=action_probabilities)


    if chosen_action == 1:
      for i in range(len(position)):
        if random.random() <0.5:
          position[i] = levy_walk(position[i]) + r1 * np.sin(r2) * np.abs(r3 * destination[i] - position[i])
          position[i] = min(num_nodes - 1, max(0, int(round(position[i]))))
          if position[i] in present_nodes:
            position[i] = random.choice(list(not_present_nodes))
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
          else:
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
        else:
          position[i] = levy_walk(position[i]) + r1 * np.cos(r2) * np.abs(r3 * destination[i] - position[i])
          position[i] = min(num_nodes - 1, max(0, int(round(position[i]))))
          if position[i] in present_nodes:
            position[i] = random.choice(list(not_present_nodes))
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
          else:
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
    else:
      for i in range(len(position)):
        if random.random() <0.5:
          position[i] = position[i] + r1 * np.sin(r2) * np.abs(r3 * destination[i] - position[i])
          position[i] = min(num_nodes - 1, max(0, int(round(position[i]))))
          if position[i] in present_nodes:
            position[i] = random.choice(list(not_present_nodes))
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
          else:
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
        else:
          position[i] = position[i] + r1 * np.cos(r2) * np.abs(r3 * destination[i] - position[i])
          position[i] = min(num_nodes - 1, max(0, int(round(position[i]))))
          if position[i] in present_nodes:
            position[i] = random.choice(list(not_present_nodes))
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
          else:
            present_nodes.add(position[i])
            not_present_nodes.remove(position[i])
    # Update probabilities based on the success or failure of chosen action
    alpha = 0.1  # Learning rate for success
    beta = 0.1   # Learning rate for failure
    action_probabilities = probabilityupdate(action_probabilities, chosen_action, signal, alpha, beta)

    return position,action_probabilities

def optimize(k,npop, max_iter, num_nodes,a):
    swarm = initialize_particle(npop,k)
    global_best_position = random.sample(range(num_nodes), k)
    global_best_fitness = calculate_influence_spread(GOT,global_best_position)
    prev_swarm = copy.deepcopy(swarm)
    action_probabilities = [0.5,0.5]
    signal = 1
    info = []
    for iter in range(max_iter):
        for i, particle in enumerate(swarm):
            position,action_probabilities = update_position(particle, global_best_position, num_nodes, iter+1, max_iter,a,action_probabilities,signal)
            swarm[i] = position
            fitness = calculate_influence_spread(GOT,position)
            if fitness > global_best_fitness:
              signal = 1
              global_best_fitness = fitness
              global_best_position = position
            else:
              signal = 0
        seed_sets = swarm + prev_swarm
        info = seed_set_evaluation(GOT, seed_sets)
        pareto_rank(info)
        crowding_distance(info)
    # Selection
    population = select_seed_sets(info, 5)
    return population




def divide_nodes_into_groups(n, m):
    if n.number_of_nodes() < m:
        raise ValueError("Number of nodes must be greater than or equal to the number of groups.")

    # Initialize an empty list of groups
    groups = [[] for _ in range(m)]

    # Assign each node to a random group
    nodes = list(n.nodes())
    random.shuffle(nodes)
    for i, node in enumerate(nodes):
        group_index = i % m  # Cycle through groups
        groups[group_index].append(node)

    return groups




def simulate_diffusion(graph, seeds):
    influnces = seeds[:]
    queue = influnces[:]
    pre_node_record = defaultdict(float) 
#     print("Queue:",queue)
#     print("Influences:",influnces)
    while len(queue) != 0:
        node = queue.pop(0)
#         print("----------------------------------------------------------------------")
#         print("Take node:",node)
#         print("Neighbour:",graph[node])
        for element in graph[node]:
            if element not in influnces:
#                 print("Element:",element,"prerecored",pre_node_record[element])
                pre_node_record[element] = pre_node_record[element] + graph[node][element]['p'] 
#                 print(pre_node_record[element])
                if pre_node_record[element] >= graph.nodes[element]['t']:
#                     print(">>>>>>>>>>>>>>>>>>node influeced:",element)
                    influnces.append(element)
                    queue.append(element)
#     influnce_num = len(influnces)
#     print("Seed set:",seeds,"Activated nodes:",influnces)
    return influnces



def logarithmic_penalty(actual_ratio, threshold_ratio, penalty_factor):
    if actual_ratio >= threshold_ratio:
        return 0  # No penalty if ratio meets or exceeds threshold
    else:
        logvalue=math.log(actual_ratio)
#         print("logvalue:",logvalue)
        return -penalty_factor * logvalue


def Calscore(threshold_ratio,actual_ratio):
    penalty_factor = 1
    penalty = logarithmic_penalty(actual_ratio, threshold_ratio, penalty_factor)
    return penalty



def findInf(IC_S,communities):
    info={}
    i=0
    for comm in communities:
        llist=[1,1]
        infnodes=set(IC_S) & set(comm)
        totalnodes=len(comm)
        llist[0]=len(infnodes)
        llist[1]=totalnodes
        info[i]=llist
        i=i+1
    return info


def balancecheck(G, activated, communities):
    IC_S=activated
    info=findInf(IC_S,communities)
    threshold=0.5
    summ=0
    final_penalty=0
    for comm in info:
        infn=info[comm][0]
        totalnodes=info[comm][1]
        actual_ratio=infn/totalnodes
        if(actual_ratio==0):
            actual_ratio=0.1/totalnodes
        pnlty=Calscore(threshold,actual_ratio)
#         print(pnlty)
        summ=summ+pnlty
#         print(summ)
    final_penalty=summ
    return final_penalty


def GetAllTheInfo(GOT,selected_seed,influenced_nodes):
    communities=groups.copy()
    community_spreads = [sum(1 for node in comm if node in influenced_nodes) for comm in communities]
    worst_case_influence = min(community_spreads)
    
    LogBased=balancecheck(GOT, influenced_nodes, communities)
    return LogBased,worst_case_influence



print("Hello")
npop = 30
max_iter = 5
num_nodes = GOT.number_of_nodes()
a = 10
# p = np.zeros((GOT.number_of_nodes(),GOT.number_of_nodes()))
groups = divide_nodes_into_groups(GOT, 10)
# print(groups)
# print("number of groups:",len(groups))
# for e in GOT.edges():
#   n = e[1]
#   if GOT.degree[n] > 0:
#     p[e[0],e[1]] = random.uniform(0,1/GOT.degree[n])
#   else:
#     p[e[0],e[1]] = random.random()

# psum = p.sum(axis=0)

for e in GOT.edges():
  GOT.edges[e]['p'] = random.uniform(0,1/GOT.degree[e[1]])

for n in GOT.nodes():
  GOT.nodes[n]['t'] = random.random()

MyFinalData={} 



# print(groups)

In [ ]:
k=10
start_time = time.time()
info=optimize(k, npop, max_iter, num_nodes,a)
end_time = time.time()  # End time
print("-------------------------------------------")
print(info)


In [ ]:

import numpy as np
selected_seed = info[-1]['seed']
print("k:",len(selected_seed))
# print("Lie:",LIE(selected_seed))
Inf=simulate_diffusion(GOT,selected_seed)
FFairness=calculate_F(GOT,selected_seed)
print("Activated nodes:",len(Inf))
# print("Fairness:",)

LogBased,Maxmin=GetAllTheInfo(GOT,selected_seed,Inf)
MyFinalData[k]={
    'k':k,
    'inf':len(Inf),
    'Maxmin':Maxmin,
    'LogBased':LogBased,
    'F_Fairness':FFairness,
    'time':end_time,
    'seedSet':selected_seed,
}
print(MyFinalData)

df = pd.DataFrame.from_dict(MyFinalData, orient='index')
excel_filename = "Amazon_"+str(k)+"_"+".xlsx"
df.to_excel(excel_filename, index=False)
print(f"Excel file '{excel_filename}' has been saved successfully.")

In [ ]:
# k=10
# info=optimize(k, npop, max_iter, num_nodes,a )
# print(info)

In [ ]:
k=20
start_time = time.time()
info=optimize(k, npop, max_iter, num_nodes,a)
end_time = time.time()  # End time

print(info)


In [ ]:

import numpy as np
selected_seed = info[-1]['seed']
print("k:",len(selected_seed))
# print("Lie:",LIE(selected_seed))
Inf=simulate_diffusion(GOT,selected_seed)
FFairness=calculate_F(GOT,selected_seed)
print("Activated nodes:",len(Inf))
# print("Fairness:",)

LogBased,Maxmin=GetAllTheInfo(GOT,selected_seed,Inf)
MyFinalData[k]={
    'k':k,
    'inf':len(Inf),
    'Maxmin':Maxmin,
    'LogBased':LogBased,
    'F_Fairness':FFairness,
    'time':end_time,
    'seedSet':selected_seed,
}
print(MyFinalData)

df = pd.DataFrame.from_dict(MyFinalData, orient='index')
excel_filename = "Amazon_"+str(k)+"_"+".xlsx"
df.to_excel(excel_filename, index=False)
print(f"Excel file '{excel_filename}' has been saved successfully.")

In [ ]:
k=30
start_time = time.time()
info=optimize(k, npop, max_iter, num_nodes,a)
end_time = time.time()  # End time

print(info)


In [ ]:

import numpy as np
selected_seed = info[-1]['seed']
print("k:",len(selected_seed))
# print("Lie:",LIE(selected_seed))
Inf=simulate_diffusion(GOT,selected_seed)
FFairness=calculate_F(GOT,selected_seed)
print("Activated nodes:",len(Inf))
# print("Fairness:",)

LogBased,Maxmin=GetAllTheInfo(GOT,selected_seed,Inf)
MyFinalData[k]={
    'k':k,
    'inf':len(Inf),
    'Maxmin':Maxmin,
    'LogBased':LogBased,
    'F_Fairness':FFairness,
    'time':end_time,
    'seedSet':selected_seed,
}
print(MyFinalData)

df = pd.DataFrame.from_dict(MyFinalData, orient='index')
excel_filename = "Amazon_"+str(k)+"_"+".xlsx"
df.to_excel(excel_filename, index=False)
print(f"Excel file '{excel_filename}' has been saved successfully.")

In [ ]:
k=40
start_time = time.time()
info=optimize(k, npop, max_iter, num_nodes,a)
end_time = time.time()  # End time

print(info)


In [ ]:

import numpy as np
selected_seed = info[-1]['seed']
print("k:",len(selected_seed))
# print("Lie:",LIE(selected_seed))
Inf=simulate_diffusion(GOT,selected_seed)
FFairness=calculate_F(GOT,selected_seed)
print("Activated nodes:",len(Inf))
# print("Fairness:",)

LogBased,Maxmin=GetAllTheInfo(GOT,selected_seed,Inf)
MyFinalData[k]={
    'k':k,
    'inf':len(Inf),
    'Maxmin':Maxmin,
    'LogBased':LogBased,
    'F_Fairness':FFairness,
    'time':end_time,
    'seedSet':selected_seed,
}
print(MyFinalData)

df = pd.DataFrame.from_dict(MyFinalData, orient='index')
excel_filename = "Amazon_"+str(k)+"_"+".xlsx"
df.to_excel(excel_filename, index=False)
print(f"Excel file '{excel_filename}' has been saved successfully.")

In [ ]:
k=50
start_time = time.time()
info=optimize(k, npop, max_iter, num_nodes,a)
end_time = time.time()  # End time

print(info)


In [ ]:

import numpy as np
selected_seed = info[-1]['seed']
print("k:",len(selected_seed))
# print("Lie:",LIE(selected_seed))
Inf=simulate_diffusion(GOT,selected_seed)
FFairness=calculate_F(GOT,selected_seed)
print("Activated nodes:",len(Inf))
# print("Fairness:",)

LogBased,Maxmin=GetAllTheInfo(GOT,selected_seed,Inf)
MyFinalData[k]={
    'k':k,
    'inf':len(Inf),
    'Maxmin':Maxmin,
    'LogBased':LogBased,
    'F_Fairness':FFairness,
    'time':end_time,
    'seedSet':selected_seed,
}
print(MyFinalData)

df = pd.DataFrame.from_dict(MyFinalData, orient='index')
excel_filename = "Amazon_"+str(k)+"_"+".xlsx"
df.to_excel(excel_filename, index=False)
print(f"Excel file '{excel_filename}' has been saved successfully.")